# Classification of low/high confidence in Iowa Gambling Task

In [1]:
import os
import pip
import sys
print(sys.executable)
#Check that we have something installed.
import nilearn
import nibabel as nib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import glob
import seaborn as sns
from nilearn.glm.first_level import FirstLevelModel
from nilearn import plotting
import pickle
from nilearn.masking import apply_mask, unmask
from nilearn import plotting
from nilearn.glm.first_level import FirstLevelModel
from scipy.stats import zscore
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from nilearn.input_data import NiftiMasker

/usr/bin/python


In [3]:
# Define the base data directory
data_dir = os.path.join(os.path.expanduser('/work/fMRI_data/BIDS_2024E'), 'derivatives')
print("Data directory:", data_dir)

# Dictionaries to store relevant files for each subject
anat_data = {}
func_data = {}

# Loop through all folders in the derivatives directory that start with "sub-XXXX" and are directories
for subject_folder in sorted(os.listdir(data_dir)):
    subject_path = os.path.join(data_dir, subject_folder)
    if subject_folder.startswith("sub-") and os.path.isdir(subject_path):  # Ensure it's a directory
        # Define the session folder path
        ses_dir = os.path.join(subject_path, 'ses-001')
        
        # Define paths for anat and func folders within each session
        anat_dir = os.path.join(ses_dir, 'anat')
        func_dir = os.path.join(ses_dir, 'func')
        
        # Store anat data paths if folder exists
        if os.path.exists(anat_dir):
            anat_data[subject_folder] = sorted(os.listdir(anat_dir))
        
        # Load functional data
        if os.path.exists(func_dir):
            # Initialize subject-specific dictionaries for func data
            func_data[subject_folder] = {}
            
            # Define paths for BOLD, brain mask, and confounds files
            bold_file = [f for f in os.listdir(func_dir) if 'desc-preproc_bold.nii.gz' in f]
            confounds_file = [f for f in os.listdir(func_dir) if 'desc-confounds_timeseries.tsv' in f]
            brain_mask_file = [f for f in os.listdir(func_dir) if f.endswith('desc-brain_mask.nii.gz')]  # Search for exact mask filename ending

            # Load confounds data if the file is present
            if confounds_file:
                confounds_path = os.path.join(func_dir, confounds_file[0])
                confounds_df = pd.read_csv(confounds_path, sep='\t')
                func_data[subject_folder]['confounds'] = confounds_df

            # Store BOLD and brain mask paths if files are present
            if bold_file:
                bold_path = os.path.join(func_dir, bold_file[0])
                func_data[subject_folder]['bold'] = bold_path
            
            if brain_mask_file:  # Check and assign brain mask if found
                brain_mask_path = os.path.join(func_dir, brain_mask_file[0])
                func_data[subject_folder]['brain_mask'] = brain_mask_path

# Print out loaded data structure for verification
print("Anatomical Data:", anat_data)
print("Functional Data:", func_data)


Data directory: /work/fMRI_data/BIDS_2024E/derivatives
Anatomical Data: {'sub-0136': ['sub-0136_ses-001_acq-T1sequence_run-1_desc-brain_mask.json', 'sub-0136_ses-001_acq-T1sequence_run-1_desc-brain_mask.nii.gz', 'sub-0136_ses-001_acq-T1sequence_run-1_desc-preproc_T1w.json', 'sub-0136_ses-001_acq-T1sequence_run-1_desc-preproc_T1w.nii.gz', 'sub-0136_ses-001_acq-T1sequence_run-1_dseg.nii.gz', 'sub-0136_ses-001_acq-T1sequence_run-1_from-MNI152NLin2009cAsym_to-T1w_mode-image_xfm.h5', 'sub-0136_ses-001_acq-T1sequence_run-1_from-T1w_to-MNI152NLin2009cAsym_mode-image_xfm.h5', 'sub-0136_ses-001_acq-T1sequence_run-1_label-CSF_probseg.nii.gz', 'sub-0136_ses-001_acq-T1sequence_run-1_label-GM_probseg.nii.gz', 'sub-0136_ses-001_acq-T1sequence_run-1_label-WM_probseg.nii.gz', 'sub-0136_ses-001_acq-T1sequence_run-1_space-MNI152NLin2009cAsym_desc-brain_mask.json', 'sub-0136_ses-001_acq-T1sequence_run-1_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz', 'sub-0136_ses-001_acq-T1sequence_run-1_space-MNI152

In [4]:
import nibabel as nib

# Load the fMRI image
fmri_img_path = func_data["sub-0136"]["bold"] 
fmri_img = nib.load(fmri_img_path)

# Get the spatial dimensions (x, y, z) and the number of time points (t)
x_voxels, y_voxels, z_voxels, time_points = fmri_img.shape

# Get voxel dimensions (in mm)
voxel_dims = fmri_img.header.get_zooms()  # Returns a tuple of (x, y, z, t) in mm and seconds

# Print the information
print(f"The fMRI images contain {x_voxels} voxels in the x-axis, {y_voxels} voxels in the y-axis, "
      f"{z_voxels} voxels in the z-axis, and {time_points} time points, with the spatiotemporal dimensions of "
      f"({voxel_dims[0]}mm, {voxel_dims[1]}mm, {voxel_dims[2]}mm, {voxel_dims[3]}s) in millimetres (mm) and seconds (s).")


The fMRI images contain 78 voxels in the x-axis, 92 voxels in the y-axis, 78 voxels in the z-axis, and 600 time points, with the spatiotemporal dimensions of (2.5260000228881836mm, 2.5260000228881836mm, 2.5mm, 1.0s) in millimetres (mm) and seconds (s).


In [5]:
# Define the base directory for events data and load behavioral data
events_data_dir = os.path.expanduser('/work/fMRI_data/BIDS_2024E')
behavioral_data = pd.read_csv("/work/SabrinaSchrollZakiHansen#5217/ACN_F24/ACN_Portfolio_2/in/detailed_confidence_data.csv")

# Initialize a dictionary to store each subject's merged DataFrame
subject_events_data = {}

# Process each subject folder to collect and merge events with behavioral data
for subject_folder in sorted(os.listdir(events_data_dir)):
    if not subject_folder.startswith("sub-"):
        continue

    # Identify subject numeric ID
    subj_numeric_id = int(subject_folder.split('-')[1])
    
    # Define path to session's functional data folder
    ses_dir = os.path.join(events_data_dir, subject_folder, 'ses-001', 'func')
    if not os.path.isdir(ses_dir):
        continue

    # Initialize a list to gather DataFrames for each session
    subject_data_list = []
    
    # Process each event file in the session directory
    for event_file in filter(lambda f: f.endswith("events.tsv"), os.listdir(ses_dir)):
        event_path = os.path.join(ses_dir, event_file)
        events_df = pd.read_csv(event_path, sep='\t')

        # Filter to include only "decks" trial type
        events_df = events_df[events_df['trial_type'] == 'decks']

        # Extract run (session) number and filter behavioral data accordingly
        run_number = int(event_file.split('_')[3].split('-')[1])
        behavioral_subset = behavioral_data[
            (behavioral_data['subject_id'] == subj_numeric_id) &
            (behavioral_data['session'] == run_number)
        ][['onset_decks', 'confidence']]
        
        # Merge events with behavioral data based on onset times
        merged_df = events_df.merge(
            behavioral_subset, how='left', left_on='onset', right_on='onset_decks'
        ).loc[:, ['onset', 'duration', 'confidence']]  # Retain only necessary columns
        
        # Add the session (run) number as a column
        merged_df['session'] = run_number

        # Map confidence to HC and LC, generate trial_type with subject ID and session
        def generate_trial_type(row, trial_idx):
            expression = "HighConf" if row['confidence'] == 2 else "LowConf"
            return f"tt_{str(trial_idx).zfill(2)}ID{subj_numeric_id}SES{run_number}STIM{expression}"

        # Apply the function to generate the trial_type for each trial in the DataFrame
        merged_df['trial_type'] = [
            generate_trial_type(row, idx) for idx, row in merged_df.iterrows()
        ]
        
        # Append this session's DataFrame to the list
        subject_data_list.append(merged_df)
    
    # Concatenate all session DataFrames for the subject
    subject_events_data[subj_numeric_id] = pd.concat(subject_data_list, ignore_index=True)

# Now subject_events_data includes session numbers and updated trial_type IDs with session information.


In [6]:
# Example
subject_events_data[136]

,onset,duration,confidence,session,trial_type
0,1.004390,5.0,2,3,tt_00ID136SES3STIMHighConf
1,15.985176,5.0,2,3,tt_01ID136SES3STIMHighConf
2,31.019258,5.0,2,3,tt_02ID136SES3STIMHighConf
3,46.019909,5.0,2,3,tt_03ID136SES3STIMHighConf
4,61.020791,5.0,1,3,tt_04ID136SES3STIMLowConf
...,...,...,...,...,...
155,526.038804,5.0,2,2,tt_35ID136SES2STIMHighConf
156,541.039531,5.0,2,2,tt_36ID136SES2STIMHighConf
157,556.040207,5.0,1,2,tt_37ID136SES2STIMLowConf
158,571.041003,5.0,2,2,tt_38ID136SES2STIMHighConf


In [ ]:
# Output directory
output_dir = '/work/SabrinaSchrollZakiHansen#5217/ACN_F24/ACN_Portfolio_2/out'
os.makedirs(output_dir, exist_ok=True)

# Loop over each subject in func_data
for participant_id, files in func_data.items():
    try:
        print(f"Processing {participant_id}...")

        # Check if all necessary files are available
        if not ('bold' in files and 'confounds' in files and 'brain_mask' in files):
            print(f"Data for participant {participant_id} is missing necessary files.")
            continue

        # Define the mask and functional image paths
        func_path = files["bold"]
        mask_path = files["brain_mask"]

        # Load the functional image and mask
        func_img = nib.load(func_path)
        mask_img = nib.load(mask_path)

        # Step 1: Normalize the functional image by dividing each voxel's time series by its mean
        func_data = func_img.get_fdata()
        mean_img_data = func_data.mean(axis=-1)
        mean_img_data[mean_img_data == 0] = 1e-8
        normalized_func_data = func_data / mean_img_data[..., np.newaxis]
        normalized_func_img = nib.Nifti1Image(normalized_func_data, func_img.affine, func_img.header)

        print("Functional image normalized.")

        # Step 2: Initialize and fit the FirstLevelModel with the normalized data
        flm_face = FirstLevelModel(
            t_r=0.7,
            slice_time_ref=0.5,
            mask_img=mask_img,
            hrf_model='glover',
            drift_model='cosine',
            high_pass=0.01,
            smoothing_fwhm=None,
            minimize_memory=True,
            noise_model='ols'
        )

        # Load event and confound data
        subj_numeric_id = int(participant_id.split('-')[1])
        events_df = subject_events_data[subj_numeric_id]
        confounds_cleaned = files['confounds'].loc[:, ['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z']]
        confounds_cleaned = confounds_cleaned.fillna(method='ffill').fillna(method='bfill')

        # Fit the model with normalized functional data
        flm_face.fit(normalized_func_img, events=events_df, confounds=confounds_cleaned)
        print(f"Model fitting completed for {participant_id}.")

        # Save the fitted model
        model_path = os.path.join(output_dir, f'{participant_id}_flm_model.pkl')
        with open(model_path, 'wb') as model_file:
            pickle.dump(flm_face, model_file)

        # Step 3: Compute and standardize contrast values
        R_face_subject = []
        for col in events_df['trial_type']:
            img = flm_face.compute_contrast(col, output_type='effect_size')
            img_data = np.squeeze(img.get_fdata())

            if img_data.shape == mask_img.shape:
                # Apply mask and standardize
                masked_data = apply_mask(nib.Nifti1Image(img_data, img.affine), mask_img)
                masked_data = zscore(masked_data)
                R_face_subject.append(masked_data)
                print(f"Applied mask and standardized contrast for trial '{col}'")
            else:
                print(f"Error: Adjusted contrast image shape {img_data.shape} does not match mask shape {mask_img.shape} for trial '{col}'")
                continue

        # Stack and save contrast data
        if R_face_subject:
            R_face_subject = np.vstack(R_face_subject)
            contrast_path = os.path.join(output_dir, f'{participant_id}_R_face.npy')
            np.save(contrast_path, R_face_subject)
            print(f"Successfully stacked and saved all trials for {participant_id} with shape: {R_face_subject.shape}")

            # Check standardized contrast statistics
            print("Standardized Contrast Mean:", np.mean(R_face_subject))
            print("Standardized Contrast Standard Deviation:", np.std(R_face_subject))

            # Save the first trial visualization as a sanity check
            img_test = unmask(R_face_subject[0, :], mask_img)
            fig = plotting.plot_glass_brain(img_test, cmap='blue_red', colorbar=True, plot_abs=False, vmin=-3, vmax=3)
            fig.savefig(os.path.join(output_dir, f'{participant_id}_first_trial_plot.png'))
            fig.close()
        else:
            print(f"No valid trials found to stack for {participant_id}.")

    except Exception as e:
        print(f"An error occurred while processing {participant_id}: {e}")


In [7]:
# Define paths
out_folder = '/work/SabrinaSchrollZakiHansen#5217/ACN_F24/ACN_Portfolio_2/out'
subject_id = "sub-0140"  # Replace with your subject ID if needed

# Load the R_face data for the subject
r_face_path = os.path.join(out_folder, f'{subject_id}_R_face.npy')
R_face = np.load(r_face_path)

In [13]:
# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import numpy as np
import os

# Define paths
out_folder = '/work/SabrinaSchrollZakiHansen#5217/ACN_F24/ACN_Portfolio_2/out'

# Load the R_face data for all subjects
all_R_face = []
all_confidence_labels = []

print("Loading R_face data for all subjects...")
num_files = len([f for f in os.listdir(out_folder) if f.endswith('_R_face.npy')])
current_file = 0
for file_name in os.listdir(out_folder):
    if file_name.endswith('_R_face.npy'):
        current_file += 1
        print(f"Processing file {current_file}/{num_files}...")
        subject_id = file_name.split('_')[0]
        r_face_path = os.path.join(out_folder, file_name)
        if os.path.exists(r_face_path):
            R_face = np.load(r_face_path)
            print(f"Loaded R_face data for {subject_id}, shape: {R_face.shape}")
            if R_face.size > 0:  # Check if the loaded array is not empty
                all_R_face.append(R_face)

                # Assuming confidence levels are binary (e.g., 0 for Low Confidence, 1 for High Confidence)
                confidence_labels = []
                for col in subject_events_data.get(int(subject_id.split('-')[1]), {}).get('confidence', []):
                    confidence_labels.append(1 if col == 2 else 0)
                all_confidence_labels.extend(confidence_labels)

print("Finished loading R_face data.")

# Combine all subjects' data if available
if all_R_face:
    # Ensure all arrays have the same number of features (voxel counts)
    min_features = min(arr.shape[1] for arr in all_R_face)
    print(f"Minimum number of features across all subjects: {min_features}")
    all_R_face = [arr[:, :min_features] for arr in all_R_face]
    all_R_face = np.vstack(all_R_face)
    all_confidence_labels = np.array(all_confidence_labels)
    print(f"Combined R_face shape: {all_R_face.shape}")
    print(f"Number of confidence labels: {len(all_confidence_labels)}")

    # Split the data into training and testing sets (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(all_R_face, all_confidence_labels, test_size=0.2, random_state=42)
    print(f"Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}")

    # Define a logistic regression model with L2 regularization
    model = make_pipeline(StandardScaler(), LogisticRegression(solver='lbfgs', max_iter=1000))
    print("Training logistic regression model...")

    # Train the model on the training data
    model.fit(X_train, y_train)
    print("Model training completed.")

    # Predict on the test data
    y_pred = model.predict(X_test)
    print("Prediction on test data completed.")

    # Calculate and print evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.2f}")
    print(f"ROC-AUC Score: {roc_auc:.2f}")
    print("Classification Report:\n", classification_report(y_test, y_pred))

    # Example of cross-validation for better model evaluation
    print("Performing cross-validation...")
    cv_scores = cross_val_score(model, all_R_face, all_confidence_labels, cv=5)
    print(f"Cross-Validation Accuracy: {np.mean(cv_scores):.2f} +/- {np.std(cv_scores):.2f}")
else:
    print("No valid R_face data found for any subject.")


Loading R_face data for all subjects...
Processing file 1/6...
Loaded R_face data for sub-0141, shape: (160, 108161)
Processing file 2/6...
Loaded R_face data for sub-0138, shape: (160, 115724)
Processing file 3/6...
Loaded R_face data for sub-0136, shape: (160, 111452)
Processing file 4/6...
Loaded R_face data for sub-0142, shape: (160, 114359)
Processing file 5/6...
Loaded R_face data for sub-0139, shape: (160, 105931)
Processing file 6/6...
Loaded R_face data for sub-0140, shape: (160, 115410)
Finished loading R_face data.
Minimum number of features across all subjects: 105931
Combined R_face shape: (960, 105931)
Number of confidence labels: 960
Training set shape: (768, 105931), Testing set shape: (192, 105931)
Training logistic regression model...


/home/ucloud/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model training completed.
Prediction on test data completed.
Accuracy: 0.56
ROC-AUC Score: 0.57
Classification Report:
               precision    recall  f1-score   support

           0       0.53      0.69      0.60        90
           1       0.62      0.45      0.52       102

    accuracy                           0.56       192
   macro avg       0.57      0.57      0.56       192
weighted avg       0.58      0.56      0.56       192

Performing cross-validation...


KeyboardInterrupt: 